In [1]:
import mindspore.nn as nn
import mindspore as ms
from mindspore import Parameter,Tensor
import numpy as np

In [2]:
# conv_weight = Tensor(np.array([[3,2,1],[2,3,2],[1,2,3]]),ms.float32).reshape(1,1,3,3)
x = Tensor(np.full((1,1,5,5), 2, np.float32))
conv = nn.Conv2d(in_channels=3,out_channels=112,kernel_size=3,pad_mode='same',has_bias=True)
bn = nn.BatchNorm2d(112)

In [32]:
bn.moving_mean

Parameter (name=mean, shape=(112,), dtype=Float32, requires_grad=False)

In [17]:
conv.bias.data.shape

(112,)

In [12]:
Tensor(conv.bias)

Tensor(shape=[1], dtype=Float32, value= [1.00000000e+001])

In [14]:
out = conv(x)

In [15]:
out

Tensor(shape=[1, 1, 5, 5], dtype=Float32, value=
[[[[2.00000000e+001, 2.60000000e+001, 2.60000000e+001, 2.60000000e+001, 1.60000000e+001],
   [2.60000000e+001, 3.80000000e+001, 3.80000000e+001, 3.80000000e+001, 2.60000000e+001],
   [2.60000000e+001, 3.80000000e+001, 3.80000000e+001, 3.80000000e+001, 2.60000000e+001],
   [2.60000000e+001, 3.80000000e+001, 3.80000000e+001, 3.80000000e+001, 2.60000000e+001],
   [1.60000000e+001, 2.60000000e+001, 2.60000000e+001, 2.60000000e+001, 2.00000000e+001]]]])

In [35]:
test = Tensor(np.arange(0,12).reshape((1,1,3,4)))

In [36]:
test

Tensor(shape=[1, 1, 3, 4], dtype=Int32, value=
[[[[ 0,  1,  2,  3],
   [ 4,  5,  6,  7],
   [ 8,  9, 10, 11]]]])

In [39]:
test.flatten()

Tensor(shape=[12], dtype=Int32, value= [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [3]:
onehot = ms.ops.OneHot()

In [6]:
label = Tensor([0,2,3,1,4])
onehot(label, 4, Tensor(1),Tensor(0))

Tensor(shape=[5, 4], dtype=Int32, value=
[[1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 0]])

In [ ]:

def prepare_targets(self, targets, anchors):
    INF = 1000000
    cls_labels = []
    reg_targets = []
    for im_i in range(len(targets)):
        targets_per_im = targets[im_i]
        assert targets_per_im.mode == "xyxy"
        bboxes_per_im = targets_per_im.bbox
        labels_per_im = targets_per_im.get_field("labels")
        anchors_per_im = cat_boxlist(anchors[im_i])
        num_gt = bboxes_per_im.shape[0];
        num_anchors_per_loc = len(self.cfg.MODEL.ATSS.ASPECT_RATIOS) * self.cfg.MODEL.ATSS.SCALES_PER_OCTAVE

        num_anchors_per_level = [len(anchors_per_level.bbox) for anchors_per_level in anchors[im_i]]
        ious = boxlist_iou(anchors_per_im, targets_per_im)

        gt_cx = (bboxes_per_im[:, 2] + bboxes_per_im[:, 0]) / 2.0
        gt_cy = (bboxes_per_im[:, 3] + bboxes_per_im[:, 1]) / 2.0
        gt_points = torch.stack((gt_cx, gt_cy), dim=1)

        anchors_cx_per_im = (anchors_per_im.bbox[:, 2] + anchors_per_im.bbox[:, 0]) / 2.0
        anchors_cy_per_im = (anchors_per_im.bbox[:, 3] + anchors_per_im.bbox[:, 1]) / 2.0
        anchor_points = torch.stack((anchors_cx_per_im, anchors_cy_per_im), dim=1)

        distances = (anchor_points[:, None, :] - gt_points[None, :, :]).pow(2).sum(-1).sqrt()

        # Selecting candidates based on the center distance between anchor box and object
        candidate_idxs = []
        star_idx = 0
        for level, anchors_per_level in enumerate(anchors[im_i]):
            end_idx = star_idx + num_anchors_per_level[level]
            distances_per_level = distances[star_idx:end_idx, :]
            topk = min(self.cfg.MODEL.ATSS.TOPK * num_anchors_per_loc, num_anchors_per_level[level])
            _, topk_idxs_per_level = distances_per_level.topk(topk, dim=0, largest=False)
            candidate_idxs.append(topk_idxs_per_level + star_idx)
            star_idx = end_idx
        candidate_idxs = torch.cat(candidate_idxs, dim=0)

        # Using the sum of mean and standard deviation as the IoU threshold to select final positive samples
        candidate_ious = ious[candidate_idxs, torch.arange(num_gt)]
        iou_mean_per_gt = candidate_ious.mean(0)
        iou_std_per_gt = candidate_ious.std(0)
        iou_thresh_per_gt = iou_mean_per_gt + iou_std_per_gt
        is_pos = candidate_ious >= iou_thresh_per_gt[None, :]

        # Limiting the final positive samples’ center to object
        anchor_num = anchors_cx_per_im.shape[0]
        for ng in range(num_gt):
            candidate_idxs[:, ng] += ng * anchor_num
        e_anchors_cx = anchors_cx_per_im.view(1, -1).expand(num_gt, anchor_num).contiguous().view(-1)
        e_anchors_cy = anchors_cy_per_im.view(1, -1).expand(num_gt, anchor_num).contiguous().view(-1)
        candidate_idxs = candidate_idxs.view(-1)
        l = e_anchors_cx[candidate_idxs].view(-1, num_gt) - bboxes_per_im[:, 0]
        t = e_anchors_cy[candidate_idxs].view(-1, num_gt) - bboxes_per_im[:, 1]
        r = bboxes_per_im[:, 2] - e_anchors_cx[candidate_idxs].view(-1, num_gt)
        b = bboxes_per_im[:, 3] - e_anchors_cy[candidate_idxs].view(-1, num_gt)
        is_in_gts = torch.stack([l, t, r, b], dim=1).min(dim=1)[0] > 0.01
        is_pos = is_pos & is_in_gts

        # if an anchor box is assigned to multiple gts, the one with the highest IoU will be selected.
        ious_inf = torch.full_like(ious, -INF).t().contiguous().view(-1)
        index = candidate_idxs.view(-1)[is_pos.view(-1)]
        ious_inf[index] = ious.t().contiguous().view(-1)[index]
        ious_inf = ious_inf.view(num_gt, -1).t()

        anchors_to_gt_values, anchors_to_gt_indexs = ious_inf.max(dim=1)
        cls_labels_per_im = labels_per_im[anchors_to_gt_indexs]
        cls_labels_per_im[anchors_to_gt_values == -INF] = 0
        matched_gts = bboxes_per_im[anchors_to_gt_indexs]

        reg_targets_per_im = self.box_coder.encode(matched_gts, anchors_per_im.bbox)
        cls_labels.append(cls_labels_per_im)
        reg_targets.append(reg_targets_per_im)

    return cls_labels,reg_targets

In [1]:
import mindspore.common.dtype as mstype
import mindspore as ms
import mindspore.nn as nn
import mindspore.numpy as mnp
from mindspore import context, Tensor
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.ops import composite as C
from mindspore.parallel._auto_parallel_context import auto_parallel_context
from mindspore.communication.management import get_group_size
from mindspore.context import ParallelMode
from mindspore import Parameter

class QualityFocalLoss(nn.Cell):
    def __init__(self, beta=2.0):
        super(QualityFocalLoss, self).__init__()
        self.sigmoid = P.Sigmoid()
        self.sigmiod_cross_entropy = P.SigmoidCrossEntropyWithLogits()
        self.pow = P.Pow()
        self.abs = P.Abs()
        self.onehot = P.OneHot()
        self.on_value = Tensor(1.0, mstype.float32)
        self.off_value = Tensor(0.0, mstype.float32)
        self.tile = P.Tile()
        self.expand_dims = P.ExpandDims()
        self.beta = beta

    def construct(self, logits, label, score):
        # print(logits[0])
        logits_sigmoid = self.sigmoid(logits)
        label = self.onehot(label, F.shape(logits)[-1], self.on_value, self.off_value)
        score = self.tile(self.expand_dims(score, -1), (1, 1, F.shape(logits)[-1]))
        label = label * score
        sigmiod_cross_entropy = self.sigmiod_cross_entropy(logits, label)
        modulating_factor = self.pow(self.abs(label - logits_sigmoid), self.beta)
        qfl_loss = sigmiod_cross_entropy * modulating_factor
        return qfl_loss

In [2]:
import numpy as np

In [9]:
np.random.seed(1)
label = Tensor(np.random.randint(0, 7 ,size=(1,10)),ms.int32)
score = Tensor(np.random.rand(1,10), ms.float32)
pred = Tensor(np.random.rand(1,10,7), ms.float32)
qfl = QualityFocalLoss()

In [10]:
qfl_out = qfl(pred,label,score)

In [11]:
qfl_out

Tensor(shape=[1, 10, 7], dtype=Float32, value=
[[[3.35580468e-001, 4.09142345e-001, 2.19223887e-001 ... 5.58021903e-001, 3.21840227e-001, 2.87731856e-001],
  [4.87759560e-001, 6.10424519e-001, 6.23453081e-001 ... 1.85243577e-001, 2.29926199e-001, 6.11671627e-001],
  [2.04597220e-001, 3.37430745e-001, 6.69921279e-001 ... 5.98448291e-002, 2.88644433e-001, 4.84150261e-001],
  ...
  [3.21578644e-002, 4.70233291e-001, 3.85300308e-001 ... 4.24804866e-001, 6.29799187e-001, 2.18185946e-001],
  [1.96005374e-001, 5.62516034e-001, 3.26141834e-001 ... 6.47379518e-001, 3.02978337e-001, 5.24947941e-001],
  [5.08965313e-001, 8.74029240e-004, 4.46266055e-001 ... 3.03490698e-001, 2.69219160e-001, 6.24374032e-001]]])